## load in data

In [1]:
relations_filename         = '../data/family_summaries.tsv'
pheno_filename             = '../data/pheno_partial.tsv'
raw_filename               = '../data/16p12_lcl_gene_reads_underscores.gct'
mapping_filename           = '../differential_expression_analysis/exon_length.tsv'


pheno                   = read.table(pheno_filename, sep='\t', header=TRUE, stringsAsFactors = F)
rownames(pheno)         = pheno$sample


relations           = read.table(relations_filename, sep='\t', header=TRUE, stringsAsFactors = F)
rownames(relations) = relations$subject


map             = read.table(mapping_filename, sep='\t', header=TRUE, stringsAsFactors = F)
rownames(map)   = map$ensembl


## GO

In [2]:
library(goseq)
suppressPackageStartupMessages(library('GenomicFeatures'))


in_dir='../family_based_analysis3/output/three_lists/'

threelists  = c('de_novo', 'inherited_from_non_carrier', 'inherited_from_carrier')

in_genes = scan('../compare_analysis_methods/in_gene_lists/edgeR.list', what=character())
gene_lengths = map[in_genes,'exons_length']

In [3]:

for (subject in rownames(relations)) {
    if (subject == 'SG011') next
    for (listype in threelists) {
        filename = paste0(in_dir, subject, '.', listype, '.tsv')
#         print(filename)
        
        de_genes = read.table(filename, header=T, stringsAsFactors = F)
        de_genes = de_genes$ensembl
        all_genes = in_genes

        genes = as.integer(all_genes%in%de_genes)
        names(genes)=in_genes

        pwf=nullp(genes, bias.data = gene_lengths)
#         pwf=nullp(genes,"hg19","ensGene")
        
        go=goseq(pwf,"hg19","ensGene")

        go$over_rep_FDR = p.adjust(go$over_represented_pvalue, method="BH")
#         go= go[go$over_rep_FDR < .05,]
        print(dim(go))

        out_dir = 'goseq_all/'
        write.table(go, paste0(out_dir, subject, '.', listype, '.tsv'), 
                    sep='\t', row.names=F)
    }
    
}